# **Import Lıbraries**

In [ ]:
import pandas as pd
import numpy as np
import librosa
import csv
import os

import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Urban_Sound/metadata/UrbanSound8K.csv")
df

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.000000,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.500000,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.500000,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.000000,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.500000,72.500000,1,5,2,children_playing
...,...,...,...,...,...,...,...,...
8727,99812-1-2-0.wav,99812,159.522205,163.522205,2,7,1,car_horn
8728,99812-1-3-0.wav,99812,181.142431,183.284976,2,7,1,car_horn
8729,99812-1-4-0.wav,99812,242.691902,246.197885,2,7,1,car_horn
8730,99812-1-5-0.wav,99812,253.209850,255.741948,2,7,1,car_horn


# **Feature Extraction**

- In the first code cell below, I am creating the dataframe which will contain **Audio Features**.
- In the second code cell, I am extracting the audio features and save them to the "**data.csv**" to able to use these features later.

In [ ]:
header = 'filename lenght chroma_stft_mean chroma_stft_var rms_mean rms_var \
          spectral_centroid_mean spectral_centroid_var spectral_bandwidth_mean spectral_bandwidth_var \
          rolloff_mean rolloff_var zero_crossing_rate_mean zero_crossing_rate_var \
          harmonic_mean harmonic_var percussive_mean percussive_var tempo flux contrast flatness'

for i in range(1, 21):
    header += f' mfcc{i}_mean'
    header += f' mfcc{i}_var'
header += ' label'
header = header.split()
file = open('data.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)

In [ ]:
genres = 'fold1 fold2 fold3 fold4 fold5 fold6 fold7 fold8 fold9 fold10 love '.split()
successed = 0

for g in genres:
    file_path = '/content/drive/MyDrive/Urban_Sound/audio/'
    for filename in os.listdir(f'{file_path}{g}'):
        full_path = f'{file_path}{g}/{filename}'
        if full_path.endswith((".wav")):
            y, sr = librosa.load(full_path, mono=True, sr=22050)

            lenght = librosa.get_duration(y=y, sr=sr)
            chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
            spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
            spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
            rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
            zcr = librosa.feature.zero_crossing_rate(y)
            mfcc = librosa.feature.mfcc(y=y, sr=sr)
            rms = librosa.feature.rms(y=y)

            harmonic, percussive = librosa.effects.hpss(y)
            tempo = librosa.feature.tempo(onset_envelope=librosa.onset.onset_strength(y=y, sr=sr), sr=sr)[0]
            flux = librosa.onset.onset_strength(y=y, sr=sr)
            contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
            flatness = librosa.feature.spectral_flatness(y=y)

            to_append = f'{filename} {lenght} \
                          {np.mean(chroma_stft)} {np.var(chroma_stft)} \
                          {np.mean(rms)} {np.var(rms)} \
                          {np.mean(spec_cent)} {np.var(spec_cent)} \
                          {np.mean(spec_bw)} {np.var(spec_bw)} \
                          {np.mean(rolloff)} {np.var(rolloff)} \
                          {np.mean(zcr)} {np.var(zcr)} \
                          {np.mean(harmonic)} {np.var(harmonic)} \
                          {np.mean(percussive)} {np.var(percussive)} \
                          {(tempo)} {np.mean(flux)} {np.mean(contrast)} {np.mean(flatness)}'

            for e in mfcc:
                to_append += f' {np.mean(e)}'
                to_append += f' {np.var(e)}'
            class_id = x = filename.split("-")
            class_label = {"0": "air_conditioner", "1": "car_horn", "2": "children_playing",
                           "3": "dog_bark", "4": "drilling", "5": "engine_idling",
                           "6": "gun_shot", "7": "jackhammer", "8": "siren",
                           "9": "street_music", "10": "love"}
            label = class_label[class_id[1]]
            to_append += f' {label}'
            file = open('data.csv', 'a', newline='')
            with file:
                writer = csv.writer(file)
                writer.writerow(to_append.split())
            print(f"\r File Name: {full_path} - Successed: {successed}", end="")
            successed +=1

 File Name: /content/drive/MyDrive/Urban_Sound/audio/love/love-10-995.wav - Successed: 9731

# **Extracted Dataframe' Sımple EDA**

In [ ]:
feature_extracted_df = pd.read_csv("/content/data.csv")
feature_extracted_df

,filename,lenght,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,rolloff_var,zero_crossing_rate_mean,zero_crossing_rate_var,harmonic_mean,harmonic_var,percussive_mean,percussive_var,tempo,flux,contrast,flatness,mfcc1_mean,mfcc1_var,mfcc2_mean,mfcc2_var,mfcc3_mean,mfcc3_var,mfcc4_mean,mfcc4_var,mfcc5_mean,mfcc5_var,mfcc6_mean,mfcc6_var,mfcc7_mean,mfcc7_var,mfcc8_mean,mfcc8_var,mfcc9_mean,mfcc9_var,mfcc10_mean,mfcc10_var,mfcc11_mean,mfcc11_var,mfcc12_mean,mfcc12_var,mfcc13_mean,mfcc13_var,mfcc14_mean,mfcc14_var,mfcc15_mean,mfcc15_var,mfcc16_mean,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,101415-3-0-2.wav,4.0,0.445801,0.100926,0.064127,0.008668,1272.322587,2.481499e+05,1929.385305,437279.862184,2479.367859,3.499473e+06,0.040970,0.000143,-2.574961e-08,0.004509,-1.309920e-05,0.005624,135.999178,1.305008,21.227080,0.003123,-402.430267,14876.565430,92.263443,4681.973145,19.303583,528.622803,-13.194186,1297.783203,-5.806573,461.490234,2.408587,103.737206,-14.885645,713.204590,-8.070933,330.622345,-9.417303,370.602875,-6.220677,185.355225,0.973379,47.285946,-5.762092,154.858109,1.656342,56.933807,0.672462,67.096268,0.964007,148.571991,5.737596,253.924896,3.124156,106.959267,3.048523,81.323135,-4.208719,80.633873,-3.609706,46.076340,dog_bark
1,102842-3-1-6.wav,4.0,0.437555,0.089910,0.005508,0.000014,2623.250506,1.420882e+05,2922.140245,68522.496105,6294.603363,1.023562e+06,0.094952,0.000594,-9.202954e-08,0.000024,4.113328e-06,0.000006,129.199219,1.040583,18.840782,0.035283,-395.932434,541.549805,81.115387,187.430618,17.608677,180.742538,21.393267,124.559952,6.876369,61.261131,11.225438,50.827663,1.630578,36.666042,5.535836,42.848858,-1.564422,29.636629,4.379283,34.740940,0.814167,16.056475,5.618686,18.897438,2.114896,21.721876,9.150099,16.073460,7.237266,29.615963,11.399624,41.565609,4.276185,19.708921,3.425487,21.119240,-4.902142,21.645121,-1.353364,14.453720,dog_bark
2,102842-3-0-1.wav,4.0,0.414020,0.085047,0.010189,0.000086,1964.691393,1.360735e+05,2735.296270,81888.481447,5128.636425,1.856378e+06,0.059692,0.000185,-2.075301e-06,0.000102,1.065492e-06,0.000022,129.199219,0.915061,19.019478,0.012199,-392.076233,171.415237,99.935471,160.706802,39.111130,73.800697,30.799818,50.763393,10.718008,43.427525,11.876123,42.958176,0.274951,31.743423,2.641112,22.724537,-5.765363,17.461048,-2.024269,14.972669,-7.755504,15.262851,-2.454248,15.169668,-4.992349,14.245397,3.564117,24.158562,3.277140,37.640675,10.335452,31.770369,5.993046,16.442160,7.624919,19.424845,-0.269627,24.004658,2.220974,16.811567,dog_bark
3,102842-3-1-0.wav,4.0,0.362982,0.091707,0.008424,0.000008,2007.009922,9.804700e+04,2656.844446,59219.164430,4974.854503,1.111185e+06,0.064216,0.000254,-7.726031e-07,0.000035,1.750105e-06,0.000016,143.554688,1.053414,19.670984,0.013854,-373.514801,478.330109,104.392410,217.515305,24.738045,131.377151,20.360533,105.808311,4.993898,72.596275,8.980869,44.973812,-3.351018,35.072128,-3.072793,50.625046,-7.168290,42.737556,2.301548,32.056007,-0.739452,31.477739,4.235912,23.053295,-0.039153,27.127007,7.930249,26.479345,8.502192,25.204235,13.688465,34.276577,5.164061,23.093395,5.025818,21.959629,-3.550596,23.107979,-1.318111,11.611815,dog_bark
4,102842-3-1-5.wav,4.0,0.414221,0.090689,0.005821,0.000012,2514.471121,1.360888e+05,2894.116626,62572.328592,6131.175064,1.056774e+06,0.089324,0.000458,1.448682e-06,0.000024,2.994112e-06,0.000007,135.999178,1.047412,18.931821,0.030127,-391.571899,502.328583,85.351158,196.966644,19.193436,151.706268,21.695240,111.186905,7.415434,53.254143,10.363002,42.792664,-0.321306,38.482327,3.408881,46.095428,-2.162839,38.081947,4.503120,40.614620,0.086177,19.633860,5.443002,21.156380,2.742051,20.576994,8.670346,19.993544,6.580069,28.996845,12.084952,31.776527,5.483204,19.552509,3.812809,17.387346,-4.512297,20.354237,-1.469074,15.576131,dog_bark
...,...

## **Missing Value**

In [ ]:
feature_extracted_df.isnull().sum().sum()

0

# **Save 10 Label Dataframe**

* I also save the original dataset features to compare the new dataset accuracy.

In [ ]:
df10_label = df.loc[df['label']!="love"]
df10_label.to_csv('data_10label.csv', index=False)